In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PyQt5.QtWidgets import QApplication, QVBoxLayout, QWidget, QLabel, QHBoxLayout, QPushButton
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtGui import QImage, QPixmap
from PyQt5 import QtCore

# Initialize global variables
bg = None

def run_avg(image, accumWeight):
    global bg
    if bg is None:
        bg = image.copy().astype("float")
        return

    cv2.accumulateWeighted(image, bg, accumWeight)

def segment(image, threshold=25):
    global bg
    diff = cv2.absdiff(bg.astype("uint8"), image)
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

def _load_weights():
    try:
        model = load_model("model_cnn.h5")
        print(model.summary())
        return model
    except Exception as e:
        return None

def getPredictedClass(model, image):
    if model is None:
        print("Model is not loaded successfully.")
        return None

    # Resize the input image to match the expected input shape of the CNN model
    resized_image = cv2.resize(image, (100, 100))

    # Preprocess the resized image
    preprocessed_image = resized_image.astype("float") / 255.0
    preprocessed_image = np.expand_dims(preprocessed_image, axis=-1)  # Add channel dimension
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)   # Add batch dimension

    # Make prediction
    prediction = model.predict(preprocessed_image)

    # Get the predicted class label
    predicted_class = np.argmax(prediction)

    classes = ["Blank", "Fist", "Ok", "Palm", "ThumbsDown", "ThumbsUp"]
    return classes[predicted_class]

gesture_to_keys = {
    "Blank": "No Key",
    "Ok": "A",
    "ThumbsUp": "S",
    "ThumbsDown": "W",
    "Fist": "D",
    "Palm": "Space"
}

gesture_to_actions = {
    "Palm": "Start",
    "ThumbsUp": "Jump",
    "ThumbsDown": "Slide",
    "Fist": "Move Right",
    "Ok": "Move Left"
}

def display_combined_feed():
    accumWeight = 0.5
    camera = cv2.VideoCapture(0)

    fps = int(camera.get(cv2.CAP_PROP_FPS))
    top, right, bottom, left = 10, 350, 225, 590
    num_frames = 0
    model = _load_weights()
    k = 0

    app = QApplication([])
    window = QWidget()
    main_layout = QVBoxLayout(window)

    # Set gradient background for the window
    # window.setStyleSheet("""
    #     QWidget {
    #         background: qlineargradient(
    #             x1: 0, y1: 0, x2: 1, y2: 1,
    #             stop: 0 #ff9a9e, stop: 1 #fad0c4
    #         );
    #     }
    # """)

    # Create a horizontal layout to hold the video label and text widget
    h_layout = QHBoxLayout()

    # Create a vertical layout to hold the video label and its title
    v_layout = QVBoxLayout()

    # Set up the title text above the video feed
    video_title = QLabel("Let's play some interesting games using hand gestures")
    video_title.setStyleSheet("font-size: 16px; font-weight: bold; color: #333;")
    v_layout.addWidget(video_title)

    # Set up the video feed
    video_label = QLabel()
    video_label.resize(200, 200)
    v_layout.addWidget(video_label)

    # Add the vertical layout to the horizontal layout
    h_layout.addLayout(v_layout)

    # Set up the text widget with instructions
    instructions = """
    <b>Gesture Controls:</b><br>
    Palm: Start (Space)<br>
    Thumbs Up: Jump (S)<br>
    Thumbs Down: Slide (W)<br>
    Fist: Move Right (D)<br>
    Ok: Move Left (A)<br>
    """
    text_label = QLabel(instructions)
    text_label.setAlignment(QtCore.Qt.AlignCenter)
    text_label.setWordWrap(True)
    text_label.setStyleSheet("""
        font-size: 22px;
        color: #555;
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 5px;
    """)
    h_layout.addWidget(text_label)

    # Add the horizontal layout to the main layout
    main_layout.addLayout(h_layout)


    # Create a quit button
    quit_button = QPushButton("Quit")
    quit_button.setStyleSheet("font-size: 18px; background-color: #ff6347; color: white;")  # Set background color to red
    quit_button.clicked.connect(window.close)  # Close the window when the button is clicked

    # Add the quit button to the main layout
    main_layout.addWidget(quit_button)

    # Set up the web page display
    web_view = QWebEngineView()
    web_view.setUrl(QUrl("https://poki.com/en/g/temple-run-2"))
    main_layout.addWidget(web_view)

    # Set the window title
    window.setWindowTitle("Interactive Gaming Control through Hand Gestures")

    window.show()

    while True:
        (grabbed, frame) = camera.read()
        frame = cv2.resize(frame, (600, 400))
        frame = cv2.flip(frame, 1)
        clone = frame.copy()
        (height, width) = frame.shape[:2]
        roi = frame[top:bottom, right:left]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successful...")
        else:
            hand = segment(gray)
            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                if k % (fps / 6) == 0:
                    cv2.imwrite('Temp.png', thresholded)
                    predictedClass = getPredictedClass(model, gray)
                    cv2.putText(clone, str(predictedClass), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    if predictedClass is not None:
                        gesture_text = gesture_to_keys.get(str(predictedClass), "Unknown Gesture")
                        cv2.putText(clone, gesture_text, (70, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    else:
                        gesture_text = gesture_to_keys.get("Blank")
                        cv2.putText(clone, gesture_text, (70, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # Update the text label with the gesture information
                    action_text = gesture_to_actions.get(str(predictedClass), "No Action")
                    text_label.setText(instructions + f"<br><b>Current Gesture:</b> {gesture_text} - {action_text}")

                thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2BGR)
                thresholded = cv2.resize(thresholded, (clone.shape[1], clone.shape[0]))  # Resize to match clone
                combined = np.hstack((clone, thresholded))

                # Draw the rectangle around the hand region
                cv2.rectangle(combined, (left, top), (right, bottom), (0, 255, 0), 2)

                # Convert the OpenCV frame to QImage
                h, w, ch = combined.shape
                bytes_per_line = ch * w
                qt_img = QImage(combined.data, w, h, bytes_per_line, QImage.Format_RGB888).rgbSwapped()

                # Convert QImage to QPixmap
                pixmap = QPixmap.fromImage(qt_img)

                # Update the QLabel with the new QPixmap
                video_label.setPixmap(pixmap)

        k += 1

        num_frames += 1

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

    app.exec_()

if __name__ == "__main__":
    display_combined_feed()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     6,554,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,649,864 (25.37 MB)

 Trainable params: 6,649,862 (25.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
[STATUS] please wait! calibrating...
[STATUS] calibration successful...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x0000024CC3C5E170>
Traceback (most recent call last):
  File "c:\Users\B.Sreevidya\AppData\Local\Programs\Python\Python310\lib\weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 